# Getting started with InstaNovo

<a target="_blank" href="https://colab.research.google.com/github/instadeepai/InstaNovo/blob/main/notebooks/getting_started_with_instanovo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In this notebook, we demo InstaNovo, a transformer neural network with the ability to translate fragment ion peaks into the sequence of amino acids that make up the studied peptide(s). We evaluate the model on the yeast test fold of nine-species dataset.

![](https://raw.githubusercontent.com/instadeepai/InstaNovo/main/graphical_abstract.jpeg)

**Paper:**

- **De novo peptide sequencing with InstaNovo: Accurate, database-free peptide identification for large scale proteomics experiments** \
  Kevin Eloff, Konstantinos Kalogeropoulos, Oliver Morell, Amandla Mabona, Jakob Berg Jespersen, Wesley Williams, Sam van Beljouw, Marcin Skwark, Andreas Hougaard Laustsen, Stan J. J. Brouns, Anne Ljungars, Erwin M. Schoof, Jeroen Van Goey, Ulrich auf dem Keller, Karim Beguir, Nicolas Lopez Carranza, Timothy P. Jenkins \
  [bioRxiv](https://www.biorxiv.org/content/10.1101/2023.08.30.555055v1), [GitHub](https://github.com/instadeepai/InstaNovo)

**Important:**

It is highly recommended to run this notebook in an environment with access to a GPU. If you are running this notebook in Google Colab:

- In the menu, go to `Runtime > Change Runtime Type > T4 GPU`

## Loading the InstaNovo model

We first install the latest instanovo from PyPi

_Note: this currently installs directly from GitHub, this will be updated in the next release._

In [ ]:

!pip install instanovo

In [ ]:
from instanovo.inference.knapsack import Knapsack
from instanovo.inference.knapsack_beam_search import KnapsackBeamSearchDecoder
from instanovo.transformer.model import InstaNovo

from tqdm import tqdm
import torch
import os
import numpy as np
import pandas as pd

We can download the model checkpoint directly from the [InstaNovo releases](https://github.com/instadeepai/InstaNovo/releases).

In [ ]:
# Download checkpoint locally
!mkdir checkpoints/
!wget -nc https://github.com/instadeepai/InstaNovo/releases/download/0.1.4/instanovo_yeast.pt -P ./checkpoints/

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

Loading the model...

In [ ]:
model, config = InstaNovo.load("./checkpoints/instanovo_yeast.pt")
model = model.to(device).eval()

## Loading the nine-species dataset
Download the [yeast test fold of the nine-species dataset](https://huggingface.co/datasets/InstaDeepAI/instanovo_ninespecies_exclude_yeast) dataset from HuggingFace.

In [ ]:
from datasets import load_dataset

# Only evaluate on a subset of the data for demo
dataset = load_dataset("InstaDeepAI/instanovo_ninespecies_exclude_yeast", split="test[:10%]")

# Otherwise evaluate on the full test set
# dataset = load_dataset("InstaDeepAI/instanovo_ninespecies_exclude_yeast", split="test")

In [ ]:
from instanovo.transformer.dataset import collate_batch
from instanovo.transformer.dataset import SpectrumDataset

s2i = {v:k for k,v in model.i2s.items()}
ds = SpectrumDataset(dataset, s2i, config["n_peaks"], return_str=True)

In [ ]:
from torch.utils.data import DataLoader

dl = DataLoader(ds, batch_size=64, shuffle=False, collate_fn=collate_batch)

In [ ]:
batch = next(iter(dl))

spectra, precursors, spectra_mask, peptides, _ = batch
spectra = spectra.to(device)
precursors = precursors.to(device)

In [ ]:
spectra.shape, precursors.shape

## Knapsack beam-search decoder

Setup knapsack beam search decoder. This may take a few minutes.

In [ ]:
def _setup_knapsack(model: InstaNovo) -> Knapsack:
    MASS_SCALE = 10000
    residue_masses = model.peptide_mass_calculator.masses
    residue_masses["$"] = 0
    residue_indices = model.decoder._aa2idx
    return Knapsack.construct_knapsack(
        residue_masses=residue_masses,
        residue_indices=residue_indices,
        max_mass=4000.00,
        mass_scale=MASS_SCALE,
    )

knapsack_path = "./checkpoints/knapsack/"

if not os.path.exists(knapsack_path):
    print("Knapsack path missing or not specified, generating...")
    knapsack = _setup_knapsack(model)
    decoder = KnapsackBeamSearchDecoder(model, knapsack)
    print(f"Saving knapsack to {knapsack_path}")
    knapsack.save(knapsack_path)
else:
    print("Knapsack path found. Loading...")
    decoder = KnapsackBeamSearchDecoder.from_file(model=model, path=knapsack_path)

## Inference time 🚀

Evaluating a single batch...

In [ ]:
with torch.no_grad():
    p = decoder.decode(
        spectra=spectra,
        precursors=precursors,
        beam_size=config["n_beams"],
        max_length=config["max_length"],
    )
preds = ["".join(x.sequence) if type(x) != list else "" for x in p]
probs = [x.log_probability if type(x) != list else -1 for x in p]

In [ ]:
from instanovo.utils.metrics import Metrics

metrics = Metrics(config["residues"], config["isotope_error_range"])

In [ ]:
aa_precision, aa_recall, peptide_recall, peptide_precision = metrics.compute_precision_recall(peptides, preds)
peptide_recall

Evaluating on the yeast test fold of the nine-species dataset:

In [ ]:
preds = []
targs = []
probs = []

for _, batch in tqdm(enumerate(dl), total=len(dl)):
    spectra, precursors, _, peptides, _ = batch
    spectra = spectra.to(device)
    precursors = precursors.to(device)

    with torch.no_grad():
        p = decoder.decode(
            spectra=spectra,
            precursors=precursors,
            beam_size=config["n_beams"],
            max_length=config["max_length"],
        )

    preds += ["".join(x.sequence) if type(x) != list else "" for x in p]
    probs += [x.log_probability if type(x) != list else -1 for x in p]
    targs += list(peptides)

In [ ]:
aa_precision, aa_recall, peptide_recall, peptide_precision = metrics.compute_precision_recall(targs, preds)
aa_error_rate = metrics.compute_aa_er(targs, preds)
auc = metrics.calc_auc(targs, preds, np.exp(pd.Series(probs)))

print(f"amino acid error rate:   {aa_error_rate}")
print(f"amino acid precision:  {aa_precision}")
print(f"amino acid recall:   {aa_recall}")
print(f"peptide precision:    {peptide_precision}")
print(f"peptide recall:  {peptide_recall}")
print(f"area under the ROC curve:  {auc}")

_Note: to reproduce the results of the paper, the entire Yeast test set should be evaluated._

Saving the predictions...

In [ ]:
pred_df = pd.DataFrame({
    "targets": targs,
    "predictions": preds,
    "log_probabilities": probs,
})
pred_df.head()

In [ ]:
pred_df.to_csv("predictions.csv", index=False)

## InstaNovo+: Iterative Refinement with a Diffusion Model
In this section, we show how to refine the predictions from the transformer model with a diffusion model.

First, we download the model checkpoint.

In [ ]:
!wget -nc https://github.com/instadeepai/InstaNovo/releases/download/0.1.5/instanovoplus_yeast.zip -P ./checkpoints/
!unzip -u ./checkpoints/instanovoplus_yeast.zip -d ./checkpoints

Next, we load the checkpoint and create a decoder object.

In [ ]:
from instanovo.diffusion.multinomial_diffusion import MultinomialDiffusion
from instanovo.inference.diffusion import DiffusionDecoder

diffusion_model = MultinomialDiffusion.load("./checkpoints/diffusion_checkpoint")
diffusion_model = diffusion_model.to(device).eval()
diffusion_decoder = DiffusionDecoder(model=diffusion_model)


Then we prepare the inference data loader using predictions from the InstaNovo transformer model.

In [ ]:
import pandas as pd
import polars as pl
from instanovo.diffusion.dataset import AnnotatedPolarsSpectrumDataset
from instanovo.diffusion.dataset import collate_batches

diffusion_dataset = AnnotatedPolarsSpectrumDataset(
    pl.from_pandas(pd.DataFrame(dataset)), peptides=preds
)

diffusion_data_loader = DataLoader(diffusion_dataset, batch_size=64, shuffle=False,
                                   collate_fn=collate_batches(
                                       residues=diffusion_model.residues,
                                       max_length=diffusion_model.config.max_length,
                                       time_steps=diffusion_decoder.time_steps,
                                       annotated=True
                                   ))

Finally, we predict sequences by iterating over the spectra and refining the InstaNovo predictions.

In [ ]:
predictions = []
log_probs = []

for batch in tqdm(diffusion_data_loader, total=len(diffusion_data_loader)):
    spectra, spectra_padding_mask, precursors, peptides, peptide_padding_mask = batch
    spectra = spectra.to(device)
    spectra_padding_mask = spectra_padding_mask.to(device)
    precursors = precursors.to(device)
    peptides = peptides.to(device)
    peptide_padding_mask = peptide_padding_mask.to(device)

    with torch.no_grad():
        batch_predictions, batch_log_probs = diffusion_decoder.decode(
            spectra=spectra,
            spectra_padding_mask=spectra_padding_mask,
            precursors=precursors,
            initial_sequence=peptides
        )
    predictions.extend(["".join(sequence) for sequence in batch_predictions])
    log_probs.extend(batch_log_probs)

Iterative refinement improves performance on this sample of the Nine Species dataset. (To replicate the performance reported in the paper, you would need to evaluate on the entire dataset.) 

In [ ]:
aa_precision_refined, aa_recall_refined, peptide_recall_refined, peptide_precision_refined = metrics.compute_precision_recall(targs, predictions=predictions)
aa_error_rate_refined = metrics.compute_aa_er(targs, predictions)
auc_refined = metrics.calc_auc(targs, predictions, np.exp(pd.Series(log_probs)))

print(f"amino acid error rate:   {aa_error_rate_refined}")
print(f"amino acid precision:  {aa_precision_refined}")
print(f"amino acid recall:  {aa_recall_refined}")
print(f"peptide precision:  {peptide_precision_refined}")
print(f"peptide recall:  {peptide_recall_refined}")
print(f"area under the ROC curve:   {auc_refined}")

In [ ]:
print(f"Decrease in AA error rate: {100*(aa_error_rate - aa_error_rate_refined):.2f}%")
print(f"Increase in AA precision: {100*(aa_precision_refined - aa_precision):.2f}%")
print(f"Increase in AA recall: {100*(aa_recall_refined - aa_recall):.2f}%")
print(f"Increase in peptide precision: {100*(peptide_precision_refined - peptide_precision):.2f}%")
print(f"Increase in peptide recall: {100*(peptide_recall_refined - peptide_recall):.2f}%")
print(f"Increase in AUC: {100*(auc_refined - auc):.2f}%")

In [ ]:
diffusion_predictions = pd.DataFrame({
    "targets": targs,
    "predictions": predictions,
    "log_probabilities": log_probs,
})
diffusion_predictions.head()

In [ ]:
diffusion_predictions.to_csv("diffusion_predictions.csv", index=False)